In [1]:
from DataPreprocessor import ProcessData
train_path = '../data/Train_Data.csv'
test_path = '../data/Test_Data.csv'

In [2]:
max_sent_len = 100
process_data = ProcessData(train_path, test_path, max_sent_len)
train_data, train_pos_data, train_label = process_data.get_train_data()
test_data, test_pos_data = process_data.get_test_data()

数据预处理...
4998
4998
4998


In [3]:
len(set(process_data.test_id))

4998

In [4]:
for item in set(process_data.testid):
    if item == '00081566':
        print(item)

00081566


In [5]:
from ModelSet import NERModel
from GetData import GetData

/Users/zhaoliang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
import pickle

obj = {}
obj['train_data'] = train_data
obj['train_pos_data'] = train_pos_data
obj['train_label'] = train_label
obj['test_data'] = test_data
obj['test_pos_data'] = test_pos_data
obj['char2id'] = process_data.char2id
obj['chunk2id'] = process_data.chunk2id
obj['id2chunk'] = process_data.id2chunk
def save_obj(obj ):
    with open('../data/save_data.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
save_obj(obj)
def load_obj():
    with open('../data/save_data.pkl', 'rb') as f:
        return pickle.load(f)

In [7]:
dic = load_obj()
train_data, train_pos_data = dic.get('train_data'), dic.get('train_pos_data')
train_label, test_data = dic.get('train_label'), dic.get('test_data')
test_pos_data, char2id = dic.get('test_pos_data'), dic.get('char2id')
chunk2id, id2chunk = dic.get('chunk2id'), dic.get('id2chunk')

In [8]:
max_sent_len = 0
for item in train_data:
    if len(item) > max_sent_len:
        max_sent_len = len(item)
for item in test_data:
    if len(item) > max_sent_len:
        max_sent_len = len(item)

In [9]:
ner = NERModel(maxlen = 100, word_dict_size = len(char2id), pos_dict_size = 57, word_vec_size = 128, class_label_count = len(chunk2id))
# ner.train2(train_data, train_label )


W0902 19:39:07.455693 4532827584 deprecation_wrapper.py:119] From /Users/zhaoliang/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0902 19:39:07.470377 4532827584 deprecation_wrapper.py:119] From /Users/zhaoliang/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0902 19:39:07.481885 4532827584 deprecation_wrapper.py:119] From /Users/zhaoliang/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0902 19:39:08.039682 4532827584 deprecation_wrapper.py:119] From /Users/zhaoliang/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_w

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_layer (Embedding)     (None, 100, 128)     746496      input_layer[0][0]                
__________________________________________________________________________________________________
zero_padding1d_1 (ZeroPadding1D (None, 104, 128)     0           embedding_layer[0][0]            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 100, 64)      41024       zero_padding1d_1[0][0]           
__________________________________________________________________________________________________
bidirectio

In [10]:
ner_result = ner.predict2(test_data, id2chunk)

W0902 19:39:08.833819 4532827584 deprecation_wrapper.py:119] From /Users/zhaoliang/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



In [11]:
new_test_data= []
for item in test_data:
    item = list(item)
    item = [process_data.id2char.get(w) for w in item]
    new_test_data.append(item)


In [12]:
import pandas as pd
table = pd.DataFrame()

for i in range(len(ner_result)):
    temp_df = pd.DataFrame(new_test_data[i]).T
    table = pd.concat([table,temp_df],ignore_index=True)
    temp_df = pd.DataFrame(ner_result[i]).T
    table = pd.concat([table,temp_df],ignore_index=True)

KeyboardInterrupt: 

In [3]:
table.to_csv('../data/test_review.csv')

NameError: name 'table' is not defined

In [13]:
import pandas as pd
test_table = pd.read_csv(test_path,dtype=object)
ids = test_table['id']

In [14]:
len(set(ids))

4998

In [15]:
for item in ids:
    if item == '00081566':
        print(item)

00081566


In [16]:
test_table[test_table['id'] == '00081566']

,id,title,text
662,00081566,熊猫量化一个千万人创业的平台,人人都知道微商是一个全新的商机，都说全民微商，但是慢慢的你会发现卖产的人，要比买产品的人多得...


In [17]:
test_table = test_table.fillna('')
data = []
for _, row in test_table.iterrows():
#     content = row['title']+row['text']
    content = row['text']
    data.append(content)

In [18]:
def get_entity_list(words_list, chunk_list):
    candidate_a = []
    a_list = []
    for i in range(len(words_list)):
        if chunk_list[i] == 'B':
            if len(candidate_a) == 0:
                candidate_a.append(words_list[i])  
            elif len(candidate_a) == 1:
                candidate_a = []
                candidate_a.append(words_list[i]) 
            else:
                a_list.append(''.join(candidate_a))
                candidate_a = []
                candidate_a.append(words_list[i]) 
        elif chunk_list[i] == 'I' and len(candidate_a)!=0 and chunk_list[i-len(candidate_a)] == 'B':
            candidate_a.append(words_list[i])
          
        else:
            if len(candidate_a) != 0:
                a_list.append(''.join(candidate_a))    
            candidate_a = []
    if len(candidate_a) != 0:
        a_list.append(''.join(candidate_a))
    return a_list

In [19]:
result = []
for i in range(len(test_data)):
    data = test_data[i]
    data = [process_data.id2char.get(w) for w in data]
    data = ''.join(data)
    a = get_entity_list(data, ner_result[i])
    result.append(';'.join(a))

In [20]:
ret = pd.DataFrame()
ret['id'] = process_data.test_id
ret['unknownEntities'] = result

In [21]:
ret.head()

,id,unknownEntities
0,83dcefb7,时空周转;时空周转;时空周转
1,83dcefb7,
2,83dcefb7,闪
3,83dcefb7,时空周转;时
4,83dcefb7,时空周转;时;时空周转;时空周转


In [22]:
new_id = []
new_result = []
groups = ret.groupby('id')
for _, g in groups:
    new_id.append(_)
    r = list(g['unknownEntities'])
    temp =set()
    for item in r:
        if item.strip() != '':
            hadflag = False
            for had in temp:
                if had in item or item in had:
                    hadflag = True
            if not hadflag:
                temp.add(item)
    
    new_result.append(';'.join(list(set(temp))))      
    

In [23]:
nret = pd.DataFrame()
nret['id'] = new_id
nret['unknownEntities'] = new_result

In [24]:
nret['unknownEntities'] = nret['unknownEntities'].apply(lambda x: ';'.join(set(str(x).split(';'))))

In [25]:
unknownEntities = list(nret['unknownEntities'])
new = []
for item in unknownEntities:
    
    item = item.split(';')
    item = list(set(item))
    item = [i for i in item if len(i.strip())>1]
    new.append(';'.join(item))
    
nret['unknownEntities'] = new

In [26]:
nret.head()

,id,unknownEntities
0,00081566,比特币;熊猫量;中海选;熊猫量化
1,000b8b75,嗨团产;嗨团团;嗨团投;嗨团平;米粒优;嗨团;嗨团有;嗨团真
2,0012d20a,人工智
3,001ac252,淘宝
4,0033ebe3,车享贷;金融;惠民生


In [27]:
def resh(x):
    temp = x.split(';')
    dic = {}
    for item in temp:
        dic[item] = len(item)
    sort = sorted(dic.items(), key = lambda x:x[1], reverse=True)
    ret = [sort[0][0]]
    
    for i in range(len(sort)-1):
        flag = False
        for item in ret:
            if sort[i+1][0] in item:
                flag = True
        if not flag:
            ret.append(sort[i+1][0])
            
    return ';'.join(ret)
                
            


In [28]:
nret['unknownEntities'] = nret['unknownEntities'].apply(resh)
nret.head()

,id,unknownEntities
0,00081566,熊猫量化;比特币;中海选
1,000b8b75,嗨团产;嗨团团;嗨团投;嗨团平;米粒优;嗨团有;嗨团真
2,0012d20a,人工智
3,001ac252,淘宝
4,0033ebe3,车享贷;惠民生;金融


In [29]:
len(set(process_data.test_id))

4998

In [30]:
nret.to_csv('../result/2019-09-02.csv', index=False)

In [25]:
for item in set(test_table['id']):
    if item not in set(nret['id']):
        print(item)

02603405
03207447
05178841
09291287
00578726
01249688
06400365
07513618
06495291
5f9b14ce
08864581
9134a045
05877500
2e338a76
09204373
04282206
00081566
8652639e
07826952
d1497ac7
